
# Prepare and install requirements for the environment:

In [1]:
!pip install stardist
!pip install csbdeep
!pip install tifffile
!pip install imagecodecs
!pip install airium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 20.2 MB/s eta 0:00:00


In [2]:
!pip freeze > requirements.txt

# Load the libraries and the images:

The image was loaded in two dictionaries, one containing the first page of the .tif file and the second one containing the second page of the .tif file.

The first dictionary contains the images in 2D of dimensions (720, 960).

The second dictionary contains the images in 3D of dimensions (120, 160, 3).

In [ ]:
import os
import os.path
from os import path
import tifffile as tiff
from tifffile import imread, TiffFile
import numpy as np
import matplotlib.pyplot as plt
from stardist.models import StarDist2D
from csbdeep.utils import normalize
from stardist.plot import render_label
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from glob import glob
from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D
from csbdeep.io import save_tiff_imagej_compatible


# Define the path to your images
image_folder = '/content/drive/MyDrive/Recruitment test/mono'

# Load the images
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.tif')]

# Load the pretrained StarDist model
model_1_ch = StarDist2D.from_pretrained('2D_versatile_fluo')

output_folder = '/content/drive/MyDrive/output_images'

# Create a directory to save the results
if path.exists(output_folder) == False:
  os.mkdir(output_folder)

np.random.seed(6)
lbl_cmap = random_label_cmap()

X = sorted(glob('/content/drive/MyDrive/Recruitment test/mono/*.tif'))
X = list(map(TiffFile,X))
dic_img_2D=dict()
dic_img_3D=dict()
for tif in X:
  img1 = normalize(tif.pages[0].asarray(), 1,99.8, (0,1))
  dic_img_2D[tif.filename]=img1
  img2 = normalize(tif.pages[1].asarray(), 1,99.8, (0,1,2))
  dic_img_3D[tif.filename]=img2


Mounted at /content/drive
Found model '2D_versatile_fluo' for 'StarDist2D'.
5320433/5320433 [==============================] - 0s 0us/step
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


# Define the functions to perform the segmentation:



*   **segmentation**:  This function performs image segmentation using a given model, normalizes the image, predicts instances, and visualizes the results.

**Parameters:**

1.   model: The segmentation model used to predict instances in the image. In this case the images were of type Fluorescence (nuclear marker) single channel the model used was: '2D_versatile_fluo'
2.   filename: The name of the image file, used for saving the output.

3.  axis_norm: The axes along which the image should be normalized. In this case, since the data was a single channel a jointly normalization was performed.

4.  c: A suffix for the output filename, useful when processing slices of an image.

5.  show_dist: A boolean indicating whether to show distances in the visualization (default is True).

*   **Process:**

1.   dic_img: dictionary with the images to be proccessed



In [ ]:
def segmentation(model, image, filename, axis_norm, c, show_dist=True):
    img = normalize(image, 1, 99.8, axis=axis_norm)
    labels, details = model.predict_instances(img)

    img_show = img if img.ndim==2 else img[...,0]
    coord, points, prob = details['coord'], details['points'], details['prob']
    segmented_img = render_label(labels)

    fig, axs = plt.subplots(1, 3, figsize=(20, 7))
    # Plotting original image
    axs[0].imshow(img_show, cmap='gray')
    axs[0].set_title('Original Image')
    axs[0].axis('off')
    # Plotting segmented image
    axs[1].imshow(segmented_img, cmap='gray')
    axs[1].set_title('Segmented Image')
    axs[1].axis('off')

    axs[2].imshow(img_show, cmap='gray')
    _draw_polygons(coord, points, prob, show_dist=show_dist)
    axs[2].set_title('Original + Segmented Image')
    axs[2].axis('off')
    plt.tight_layout()

    # Save figures
    base_name = filename.replace('.tif', '')

    plt.savefig(os.path.join(output_folder, f'{base_name}_output{c}.png'))
    plt.cla()
    fig.clf()
    plt.close('all')

def process_images(dic_img):

  for key in dic_img:
    img=dic_img[key]
    axis_norm = (0,1)
    model=model_1_ch
    if img.ndim == 2:  # single channel
        c = ''
        segmentation(model, img, key, axis_norm, c, show_dist=True)

    elif img.ndim == 3 and img.shape[-1] == 3:
        segmented_slices = []
        for i in range(img.shape[2]):
          im = img[:, :, i]
          c = f'_slide_{i}'
          segmentation(model, im, key, axis_norm, c, show_dist=True)

    else:
        raise ValueError(f"Unexpected image dimensions: {img.shape}")


# Run the functions to perform the segmentation

*   **Highlight:** The second dictionary containing 3D images was processed using the same model as the 2D images. Each slide of the 3D images was iterated through to perform segmentation individually. However, since the existing 3D models were trained on volumes with more slides (>3), the segmentation output did not meet expectations. One alternative approach could be to retrain the model using the current dataset and apply augmentation techniques. Nonetheless, considering the limited size of the training data, there is a high risk of overfitting.




In [ ]:
process_images(dic_img_2D)
process_images(dic_img_3D)

# Create the HTML report and ave it at the output folder

In [ ]:
# Get list of image files in the folder
image_files = [f for f in os.listdir(output_folder) if f.endswith('.png')]

# Read the HTML template
with open(output_folder+'/index_template.html', 'r') as file:
    html_template = file.read()

# Define the HTML structure for the image slides
image_slide_template = """
<div data-thq="slider-slide" class="gallery8-slider-slide swiper-slide">
  <img alt="{alt_text}" src="{src}" class="gallery8-image thq-img-ratio-4-3" />
</div>
"""

# Generate the new image slides
new_image_slides = ""
for image in image_files:
    image_path = os.path.join('.', image)
    new_image_slides += image_slide_template.format(alt_text=image, src=image_path)

# Replace the placeholder in the template with the new image slides
new_html = html_template.replace(
    '<div data-thq="slider-wrapper" class="swiper-wrapper">',
    f'<div data-thq="slider-wrapper" class="swiper-wrapper">{new_image_slides}'
)

# Save the modified HTML to a new file
file_report = '/index.html'
with open(output_folder+file_report, 'w') as file:
    file.write(new_html)

print(f"Updated HTML file has been saved to {output_folder+file_report}")


Updated HTML file has been saved to /content/drive/MyDrive/output_images/index.html
